Primer Job Formater

In [26]:
from MRE import Job
inputDir = "./jugadores/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [27]:
def fmap(key, value, context):
    lista = value.split()
    j_retador = int(key)
    j_retado = int(lista[0])
    context.write((j_retado,'RETADO'), j_retador)
def fmap2(key, value, context):
    lista = value.split()
    context.write((int(key), 'PP')  , ((float(lista[0]) + 1)/(float(lista[1])+1)))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1

def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP es el primero 
        return -1
    else:
        return 1 #Caso retado

def fred(key, values, context):
    # El primer valor DEBE ser el PP del jugador J por el cmpShuffle
    pp_jugador_j = float(values.next())
    
    # Iteramos sobre los retadores (los 'v' son los ID de los jugadores J)
    for id_retador in values:
        # Enviamos al retador la información de su retado:
        # (ID_Retador, (PH_del_retado, PP_del_retado))
        context.write(id_retador, (puntaje_heroico_inicial, pp_jugador_j))

    
puntaje_heroico_inicial = 1

jobCount = Job(inputDir, outputDir, fmap, fred)
jobCount.setSortCmp(cmpShuffle)
jobCount.setShuffleCmp(cmpShort)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(promedioDir,fmap2)

success = jobCount.waitForCompletion()

print(success)

True


Segundo job iterativo

In [28]:
inputDir = "./format/"
outputDir = "./format/"

In [29]:
def fmap(key, value, context):
    lista = value.split("\t")
    context.write((key,"ESTRUCTURA"),lista)
    set_limpio = lista[1].strip("{} ").split(",")
    if set_limpio[0] != '':
        for v in set_limpio:
            v = v.strip()
            context.write((v, "CONTRIBUCION"), (lista[0],lista[-1]))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1
    
def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "ESTRUCTURA"): #PP y PH es el primero 
        return -1
    else:
        return 1 #Caso Contribucion

def fred(key, values, context):
    # 'values' es un ValuesIterator
    estructura = next(values) 
    # Estructura esperada: [PP_i, {lista_de_retados}, PH_viejo]
    
    pp_i = float(estructura[0])
    ph_viejo = float(estructura[-1])
    retados = estructura[1]
    
    acumulador = 0
    for v in values:
        # v viene del Mapper como (PP_j, PH_j)
        pp_j = float(v[0])
        ph_j = float(v[1])
        acumulador += ph_j * (pp_i / pp_j)
    
    nuevo_ph = (alfa * acumulador) + (1 - alfa)
    
    # Emitimos manteniendo la estructura para la siguiente iteración
    # Formato: ID \t PP \t {Retados} \t PH_Nuevo
    context.write(key[0], [pp_i, retados, nuevo_ph, ph_viejo])



# JOB INTERMEDIO QUE SOLO CALCULA EL MAE ENTRE DOS ITERACIONES
def map_mae(key, value, context):
    # Asumiendo que la estructura de value es: [PP, {Retados}, PH_Nuevo, PH_Viejo]
    # Si 'value' viene de disco como string, hay que parsearlo
    datos = value.split("\t")
        
    ph_nuevo = float(datos[2])
    ph_viejo = float(datos[3])
    
    # Calculamos el error absoluto: |PH_nuevo - PH_viejo|
    error_absoluto = abs(ph_nuevo - ph_viejo)
    
    # Enviamos a una clave fija para que un Reducer calcule el promedio final
    # Emitimos (error, 1) para poder calcular el promedio (Suma / Total)
    context.write("GLOBAL_MAE", (error_absoluto, 1))

def red_mae(key, values, context):
    suma_error_absoluto = 0
    conteo_jugadores = 0
    
    for err, count in values:
        suma_error_absoluto += err
        conteo_jugadores += count
    
    # MAE = Suma de errores absolutos / N
    mae_final = suma_error_absoluto / conteo_jugadores if conteo_jugadores > 0 else 0
    
    context.write("RESULTADO_MAE", mae_final)


# Parámetros iniciales
alfa = 0.1
error_tolerado = 0.001
max_iter = 20
input_dir = "./format/" # Salida del Job anterior (ID \t [PP, PH, {Retados}])

for i in range(max_iter):
    output_dir = f"./iteracion_{i}/"
    
    # Configuramos el Job
    job = Job(input_dir, output_dir, fmap, fred)
    job.setParams(alfa)
    job.setShuffleCmp(cmpShort)
    job.setSortCmp(cmpShuffle)
    
    success = job.waitForCompletion()
    
    # --- CÁLCULO DE CONVERGENCIA (MSE) ---
    mae_dir = f"./mae_iter_{i}/"
    
    # 1. Ejecutar Job de cálculo de PH (el que ya tienes)
    # 2. Ejecutar Job de MAE
    jobMAE = Job(output_dir, mae_dir, map_mae, red_mae)
    jobMAE.waitForCompletion()
    
    # 3. Leer el resultado del MAE desde el Driver
    # (El driver solo lee un archivo de texto de pocos bytes)
    try:
        with open(f"{mae_dir}/output.txt", "r") as f:
            # Formato esperado: RESULTADO_MAE \t 0.00045
            linea = f.read().split("\t")
            mae_actual = float(linea[1])
            
        print(f">>> Iteración {i} - MAE: {mae_actual:.6f}")
        
        if mae_actual < error_tolerado:
            print(f"Convergió en iteración {i} con MAE {mae_actual}")
            break
            
    except Exception as e:
        print(f"Error leyendo MAE: {e}")
        break
    
    # El output de esta vuelta es el input de la siguiente
    input_dir = output_dir

print(success)

>>> Iteración 0 - MAE: 1257.551749
>>> Iteración 1 - MAE: 1812.838400
>>> Iteración 2 - MAE: 1812.838400
>>> Iteración 3 - MAE: 1812.838400
>>> Iteración 4 - MAE: 1812.838400
>>> Iteración 5 - MAE: 1812.838400
>>> Iteración 6 - MAE: 1812.838400
>>> Iteración 7 - MAE: 1812.838400
>>> Iteración 8 - MAE: 1812.838400
>>> Iteración 9 - MAE: 1812.838400
>>> Iteración 10 - MAE: 1812.838400
>>> Iteración 11 - MAE: 1812.838400
>>> Iteración 12 - MAE: 1812.838400
>>> Iteración 13 - MAE: 1812.838400
>>> Iteración 14 - MAE: 1812.838400
>>> Iteración 15 - MAE: 1812.838400
>>> Iteración 16 - MAE: 1812.838400
>>> Iteración 17 - MAE: 1812.838400
>>> Iteración 18 - MAE: 1812.838400
>>> Iteración 19 - MAE: 1812.838400
True


Tercer Job saca top 10 

In [30]:
inputDir = "./format/"
outputDir = "./out/"

In [31]:
import heapq

def fmap(key, value, context):
    lista = value.split("\t")
    context.write(1, (key, float(lista[-1])))

def fcombiner(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(key, (clave,maximo))

def fred(key, values, context):
    # simple combiner: sum counts locally
    top_maximo = []
    
    for v in values:
        elemento = (v[1],v[0])
        if len(top_maximo) < 10:
            # Si hay menos de 10 elementos, simplemente lo añadimos al heap.
            heapq.heappush(top_maximo, elemento)
        else:
            # Si el nuevo score (elemento[0]) es MAYOR que el elemento MÁS PEQUEÑO 
            # del heap (top_maximo[0]), reemplazamos el pequeño.
            if elemento[0] > top_maximo[0][0]:
                heapq.heapreplace(top_maximo, elemento)
    top_10_final = sorted(top_maximo, key=lambda e: e[0], reverse=True)
    for (maximo, clave) in top_10_final:
        context.write(clave, maximo)

jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCountBoth.setCombiner(fcombiner)
success = jobCountBoth.waitForCompletion()

print(success)

True
